# *Streptococcus pneumoniae* R6

### NCBI (*pbp1a*, *pbp2b* e *pbp2x*)

Escolheu-se a estirpe R6 de Streptococcus pneumoniae uma vez que é um genoma de referência utilizado por investigadores e mencionados nos seus artigos.
Para proceder à análise dos genes de interesse (*pbp1a*, *pbp2b* e *pbp2x*) nesta estirpe, foi primeiro necessário procurar o seu respetivo identificador (que se descobriu ser **AE007317.1**). Desta forma, estamos em condições para analisar os genes de interesse.<br><br>

Começamos por extrair:
- Alguma informação geral da estirpe R6
- A localização dos respetivos genes do seu genoma
- As sequências dos genes
- Os identificadores das respetivas proteínas
- As respetivas sequências proteicas

In [3]:
from Bio import Entrez
from Bio import SeqIO

genes_int = ["pbpX","pbpA","pbp2b"]
local = {}
gene_seq = {}
prot_id = {}
prot = {}

Entrez.email = "pg45474@alunos.uminho.pt"
handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id="AE007317.1")
seq_record = SeqIO.read(handle, "genbank")

################################
print("Título:",seq_record.description)
print("ID GenBank (e outros IDs):",seq_record.annotations["accessions"])
print("Taxonomia:",seq_record.annotations["taxonomy"])
print("Tipo de molécula:",seq_record.annotations["molecule_type"])
print("Topologia:",seq_record.annotations["topology"])
print("Referências externas:",seq_record.dbxrefs)
print()
################################

for s in seq_record.features:
    if s.type == "CDS":
        qual = dict(s.qualifiers)
        
        if "gene" in qual:
            if qual["gene"][0] in genes_int:
                loc = s.location
                #########################
                print("Gene:",qual["gene"][0])
                print("Produto:", qual["product"])
                print()
                #########################
                local[qual["gene"][0]] = (int(loc.start), int(loc.end), loc.strand)
                gene_seq[qual["gene"][0]] = s.extract(seq_record.seq)
                prot_id[qual["gene"][0]] = qual["protein_id"]
                prot[qual["gene"][0]] = qual["translation"]

handle.close()

Título: Streptococcus pneumoniae R6, complete genome
ID GenBank (e outros IDs): ['AE007317', 'AE008385-AE008568']
Taxonomia: ['Bacteria', 'Firmicutes', 'Bacilli', 'Lactobacillales', 'Streptococcaceae', 'Streptococcus']
Tipo de molécula: DNA
Topologia: circular
Referências externas: ['BioProject:PRJNA278', 'BioSample:SAMN02603218']

Gene: pbpX
Produto: ['Penicillin-binding protein 2X']

Gene: pbpA
Produto: ['Penicillin-binding protein 1A']

Gene: pbp2b
Produto: ['Penicillin-binding protein 2B']



Já temos os dados principais para começar o estudo dos nossos genes. Para facilitar a análise, vamos guardar os dados em 6 ficheiros FASTA (3 contendo as sequências dos 3 genes, e 3 contendo as sequências das proteínas dos respetivos genes). Na primeira linha (de anotação) vamos escrever:
- Os nomes dos genes
- O início da sequência relativamente ao genoma
- O fim da sequência relativamente ao genoma
- A cadeia onde se encontra no genoma
- Os identificadores das respetivas proteínas

In [2]:
from Bio import SeqIO

genes_int = ["pbpX","pbpA","pbp2b"]
#local
#gene_seq
#prot_id
#prot


for g in genes_int:
    ####################
    print("Gene:",g)
    print("Início:",local[g][0])
    print("Fim:",local[g][1])
    print("Cadeia:",local[g][2])
    print("Sequência DNA:",gene_seq[g][0:75]+"...")
    print("ID Proteína:",prot_id[g][0])
    print("Sequência Proteica:",prot[g][0][0:75]+"...")
    print("----------")
    ####################
    inicio, fim, strand = local[g]
    annot = ";".join([g,str(inicio),str(fim),str(strand),prot_id[g][0]])
    
    file = open(g+".fasta","w")
    file.write(">"+annot+"\n"+str(gene_seq[g]))
    file.close()
    
    file = open(g+"_"+prot_id[g][0]+".fasta","w")
    file.write(">"+annot+"\n"+prot[g][0])
    file.close()



Gene: pbpX
Início: 302260
Fim: 304513
Cadeia: 1
Sequência DNA: ATGAAGTGGACAAAAAGAGTAATCCGTTATGCGACCAAAAATCGGAAATCGCCGGCTGAAAACAGACGCAGAGTT...
ID Proteína: AAK99108.1
Sequência Proteica: MKWTKRVIRYATKNRKSPAENRRRVGKSLSLLSVFVFAIFLVNFAVIIGTGTRFGTDLAKEAKKVHQTTRTVPAK...
----------
Gene: pbpA
Início: 332862
Fim: 335022
Cadeia: -1
Sequência DNA: ATGAACAAACCAACGATTCTGCGCCTAATCAAGTATCTGAGCATTAGCTTCTTAAGCTTGGTTATCGCAGCCATT...
ID Proteína: AAK99133.1
Sequência Proteica: MNKPTILRLIKYLSISFLSLVIAAIVLGGGVFFYYVSKAPSLSESKLVATTSSKIYDNKNQLIADLGSERRVNAQ...
----------
Gene: pbp2b
Início: 1494215
Fim: 1496273
Cadeia: -1
Sequência DNA: ATGAGACTGATTTGTATGAGAAAATTTAACAGCCATTCGATTCCGATTCGGCTTAATTTATTGTTTTCAATCGTC...
ID Proteína: AAL00321.1
Sequência Proteica: MRLICMRKFNSHSIPIRLNLLFSIVILLFMTIIGRLLYMQVLNKDFYEKKLASASQTKITSSSARGEIYDASGKP...
----------


Tendo obtido as sequências genéticas e respetivas sequências proteicas dos genes em estudo, estamos em condições de utilizálos para, nomeadamente:
- Procurar sequências homólogas (com o algorítmo **BLAST**)
- Extrair alguma informação sobre cada proteína da base de dados **UniProt**

# *pbp1a*

### BLAST

Vamos correr um BLAST contra a sequência do gene pbp1a para encontrar sequências homólogas

In [3]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW



files = [("pbpA","blastn"),("pbpA_AAK99133.1","blastp")]

for f,d in files:
    record = SeqIO.read(open(f"{f}.fasta"), format="fasta")
    result_handle = NCBIWWW.qblast(d, "nr", record.format("fasta"), expect=0.05, hitlist_size=100)

    save_file = open(f"{f}_blast.xml", "w")
    save_file.write(result_handle.read())

    save_file.close()
    result_handle.close()

In [ ]:
#pode-se alterar mais parâmetros:
from Bio.Blast import NCBIWWW
help(NCBIWWW.qblast)

Já temos os resultados do BLAST para a sequência nucleotídica e proteica. Vamos agora efetuar os alinhamentos entre a sequência query (pbp1a) com as sequências homólogas de forma percetível e guardá-las em ficheiros ("pbpA_aligns.xml" para a sequência do gene, e "pbpA_AAK99133.1_aligns.xml" para a sequência da proteína)

In [1]:
from Bio.Blast import NCBIXML
import re

files = [("pbpA",{}),("pbpA_AAK99133.1",{})]

for f in files:
    output1 = open(f"{f[0]}_aligns.xml", "w")

    result_handle = open(f"{f[0]}_blast.xml")

    blast_records = NCBIXML.parse(result_handle)

    for record in blast_records:

        first = ">QUERY\n"+record.alignments[0].hsps[0].query+"\n\n"
        f[1]["query"] = first
        
        for align in record.alignments:

            for hsp in align.hsps:

                match = re.sub(r"\w", "|", hsp.match)
                ###################
                #print ('****Alignment****')
                #print ('sequence:', align.title)
                #print ('ID:', align.hit_id)
                #print ('length:', align.length)
                #print ('e value:', hsp.expect)
                #print (hsp.query[0:30] + '...' + hsp.query[-30:] )
                #print( match[0:30] + '...' + match[-30:] )
                #print( hsp.sbjct[0:30] + '...' + hsp.sbjct[-30:] )
                #print("\n\n\n")
                ###################
                output1.write('****Alignment****\n')
                output1.write('sequence: '+ align.title+"\n")
                output1.write('ID: '+ align.hit_id+"\n")
                output1.write('length: '+ str(align.length)+"\n")
                output1.write('e value: '+ str(hsp.expect)+"\n")

                try:
                    try:
                        write = re.search(r"(gb|emb|sp|dbj)\|.+?\|", align.title).group()
                    except:
                        write = re.findall(r"\s.+?,", align.title)[0][1:-1].strip()
                        
                    f[1][write] = ">"+write+"\n"+hsp.sbjct+"\n\n"
                    
                except:
                    try:
                        write = re.search(r"(gb|emb|sp|dbj)\|.+?\|", align.title).group()
                    except:
                        write = re.findall(r"\s.+", align.title)[0][1:].strip()
                        
                    f[1][write] = ">"+write+"\n"+hsp.sbjct+"\n\n"

                st = 0
                for l in range(int(len(hsp.query)/100)+1):
                    position = st+100 if st+100<=len(hsp.query) else len(hsp.query)
                    output1.write(hsp.query[st:st+100]+f" {position}"+"\n")
                    output1.write(match[st:st+100]+"\n")
                    output1.write(hsp.sbjct[st:st+100]+"\n")
                    st += 100
                output1.write("\n\n")

    output1.close()
    result_handle.close()

for f in files:
    output2 = open(f"{f[0]}_homo.fasta", "w")
    order = list(f[1])
    order.sort()
    for o in order:
        if o == order[-1]:
            output2.write(re.sub(r"\ ", "_", f[1][o].strip()))
        else:
            output2.write(re.sub(r"\ ", "_", f[1][o]))
    output2.close()
    

#Ás vezes tem espaço, às vezes tem "+", PORQUÊ????

O "output1" gera os ficheiros com os alinhamentos (1 com a sequência de DNA, outro com a sequência da proteína). O "output2" gera 2 ficheiros (na mesma lógica do que o "output1") apenas com as sequências homólogas encontradas em formato multi-FASTA.

De seguida, vamos voltar a focarmo-nos nas proteínas em estudo. Para obter informações sobre estas proteínas, podemos recorrer ao site da **UniProt**.<br>

### UniProt

Primeiro precisamos de transferir o ficheiro correspondente à proteína de interesse em formato txt (fez-se manualmente). De seguida, já estamos em condições de processar esse ficheiro:

In [8]:
from Bio import ExPASy
import Bio.SwissProt as sp

with open("swissprot_pbp1a.txt") as handle:
    record = sp.read(handle)
    print("Nome:", record.entry_name,"\n")
    for acc in record.accessions:
        print("ID UniProt:", acc)
    print()
    for des in record.description.split(";"):
        print(des)
    print("Organismo:",repr(record.organism),"\n")
    print("Classificao do organismo:",record.organism_classification,"\n")
    print(record.sequence[:20] + "...","\n")
    print("Tamanho da seq:", record.sequence_length,"\n")
    print("Palavras Chave:",record.keywords,"\n")
    
counter = 1
for r in record.features:
    if r.type == "ACT_SITE":
        print(f"Sítio ativo {counter}:",r.location)
        counter += 1

Nome: PBPA_STRR6 

ID UniProt: Q8DR59

RecName: Full=Penicillin-binding protein 1A
 Short=PBP-1A
 AltName: Full=Exported protein 2
 Includes: RecName: Full=Penicillin-insensitive transglycosylase
 EC=2.4.1.129 {ECO:0000250|UniProtKB:P02918}
 AltName: Full=Peptidoglycan TGase
 Includes: RecName: Full=Penicillin-sensitive transpeptidase
 EC=3.4.16.4 {ECO:0000250|UniProtKB:P02918}
 AltName: Full=DD-transpeptidase

Organismo: 'Streptococcus pneumoniae (strain ATCC BAA-255 / R6).' 

Classificao do organismo: ['Bacteria', 'Firmicutes', 'Bacilli', 'Lactobacillales', 'Streptococcaceae', 'Streptococcus'] 

MNKPTILRLIKYLSISFLSL... 

Tamanho da seq: 719 

Palavras Chave: ['3D-structure', 'Antibiotic resistance', 'Carboxypeptidase', 'Cell shape', 'Cell wall biogenesis/degradation', 'Glycosyltransferase', 'Hydrolase', 'Multifunctional enzyme', 'Peptidoglycan synthesis', 'Protease', 'Reference proteome', 'Secreted', 'Transferase'] 

Sítio ativo 1: [90:91]
Sítio ativo 2: [369:370]


As informações relativamente aos nomes, identificadores e sequência da proteína, mais as informações acerca da espécie e estirpe ao qual pertence a proteína, já tinhamos extraído do NCBI.
Para além destas, o PDB permite extrair informação acerca dos sítios ativos de cada proteína. Neste caso, vemos que tem 2 sítios ativos, indicando a sua localização no gene.<br>
Sabemos pela literatura que o primeiro sítio ativo [90:91] é o da **transglicosilase**, e o segundo [369:370] é o da **transpeptidase**.

# *pbp2b*

### BLAST

Vamos correr um BLAST contra a sequência do gene pbp2b para encontrar sequências homólogas

In [5]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW



files = [("pbp2b","blastn"),("pbp2b_AAL00321.1","blastp")]

for f,d in files:
    record = SeqIO.read(open(f"{f}.fasta"), format="fasta")
    result_handle = NCBIWWW.qblast(d, "nr", record.format("fasta"), expect=0.05, hitlist_size=100)

    save_file = open(f"{f}_blast.xml", "w")
    save_file.write(result_handle.read())

    save_file.close()
    result_handle.close()

Com os resultados do BLAST, vamos agora efetuar os alinhamentos entre a sequência query (pbp2b) com as sequências homólogas de forma percetível e guardá-las em ficheiros ("pb2b_aligns.xml" para a sequência do gene, e "pbp2b_AAL00321.1_aligns.xml" para a sequência da proteína)

In [2]:
from Bio.Blast import NCBIXML
import re

files = [("pbp2b",{}),("pbp2b_AAL00321.1",{})]

for f in files:
    output1 = open(f"{f[0]}_aligns.xml", "w")

    result_handle = open(f"{f[0]}_blast.xml")

    blast_records = NCBIXML.parse(result_handle)

    for record in blast_records:

        first = ">QUERY\n"+record.alignments[0].hsps[0].query+"\n\n"
        f[1]["query"] = first
        
        for align in record.alignments:

            for hsp in align.hsps:

                match = re.sub(r"\w", "|", hsp.match)
                ###################
                #print ('****Alignment****')
                #print ('sequence:', align.title)
                #print ('ID:', align.hit_id)
                #print ('length:', align.length)
                #print ('e value:', hsp.expect)
                #print (hsp.query[0:30] + '...' + hsp.query[-30:] )
                #print( match[0:30] + '...' + match[-30:] )
                #print( hsp.sbjct[0:30] + '...' + hsp.sbjct[-30:] )
                #print("\n\n\n")
                ###################
                output1.write('****Alignment****\n')
                output1.write('sequence: '+ align.title+"\n")
                output1.write('ID: '+ align.hit_id+"\n")
                output1.write('length: '+ str(align.length)+"\n")
                output1.write('e value: '+ str(hsp.expect)+"\n")

                try:
                    try:
                        write = re.search(r"(gb|emb|sp|dbj)\|.+?\|", align.title).group()
                    except:
                        write = re.findall(r"\s.+?,", align.title)[0][1:-1].strip()
                        
                    f[1][write] = ">"+write+"\n"+hsp.sbjct+"\n\n"
                    
                except:
                    try:
                        write = re.search(r"(gb|emb|sp|dbj)\|.+?\|", align.title).group()
                    except:
                        write = re.findall(r"\s.+", align.title)[0][1:].strip()
                        
                    f[1][write] = ">"+write+"\n"+hsp.sbjct+"\n\n"

                st = 0
                for l in range(int(len(hsp.query)/100)+1):
                    position = st+100 if st+100<=len(hsp.query) else len(hsp.query)
                    output1.write(hsp.query[st:st+100]+f" {position}"+"\n")
                    output1.write(match[st:st+100]+"\n")
                    output1.write(hsp.sbjct[st:st+100]+"\n")
                    st += 100
                output1.write("\n\n")

    output1.close()
    result_handle.close()

for f in files:
    output2 = open(f"{f[0]}_homo.fasta", "w")
    order = list(f[1])
    order.sort()
    for o in order:
        if o == order[-1]:
            output2.write(re.sub(r"\ ", "_", f[1][o].strip()))
        else:
            output2.write(re.sub(r"\ ", "_", f[1][o]))
    output2.close()

O "output1" gera os ficheiros com os alinhamentos (1 com a sequência de DNA, outro com a sequência da proteína). O "output2" gera 2 ficheiros (na mesma lógica do que o "output1") apenas com as sequências homólogas encontradas em formato multi-FASTA.

### UniProt

Após ter transferido o ficheiro correspondente à proteína de interesse em formato txt, podemos processar esse ficheiro:

In [7]:
from Bio import ExPASy
import Bio.SwissProt as sp

with open("swissprot_pbp2b.txt") as handle:
    record = sp.read(handle)
    print("Nome:", record.entry_name,"\n")
    for acc in record.accessions:
        print("ID UniProt:", acc)
    print()
    for des in record.description.split(";"):
        print(des)
    print("Organismo:",repr(record.organism),"\n")
    print("Classificao do organismo:",record.organism_classification,"\n")
    print(record.sequence[:20] + "...","\n")
    print("Tamanho da seq:", record.sequence_length,"\n")
    print("Palavras Chave:",record.keywords,"\n")
    
    
counter = 1
for r in record.features:
    if r.type == "ACT_SITE":
        print(f"Sítio ativo {counter}:",r.location)
        counter += 1

Nome: PBP2_STRR6 

ID UniProt: P0A3M6
ID UniProt: P10524

RecName: Full=Penicillin-binding protein 2B

Organismo: 'Streptococcus pneumoniae (strain ATCC BAA-255 / R6).' 

Classificao do organismo: ['Bacteria', 'Firmicutes', 'Bacilli', 'Lactobacillales', 'Streptococcaceae', 'Streptococcus'] 

MRKFNSHSIPIRLNLLFSIV... 

Tamanho da seq: 680 

Palavras Chave: ['3D-structure', 'Antibiotic resistance', 'Cell membrane', 'Cell shape', 'Cell wall biogenesis/degradation', 'Membrane', 'Peptidoglycan synthesis', 'Reference proteome', 'Transmembrane', 'Transmembrane helix'] 

Sítio ativo 1: [385:386]


A UniProt aponta para 1 sítio ativo, indicando a sua localização no gene na posição [385:386], que, pela literatura, sabemos que é da sua atividade de **transpeptidase**.

# *pbp2x*

Vamos correr um BLAST contra a sequência do gene pbp2x para encontrar sequências homólogas

In [ ]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW



files = [("pbpX","blastn"),("pbpX_AAK99108.1","blastp")]

for f,d in files:
    record = SeqIO.read(open(f"{f}.fasta"), format="fasta")
    result_handle = NCBIWWW.qblast(d, "nr", record.format("fasta"), expect=0.05, hitlist_size=100)

    save_file = open(f"{f}_blast.xml", "w")
    save_file.write(result_handle.read())

    save_file.close()
    result_handle.close()

Com os resultados do BLAST, vamos agora efetuar os alinhamentos entre a sequência query (pbp2b) com as sequências homólogas de forma percetível e guardá-las em ficheiros ("pb2x_aligns.xml" para a sequência do gene, e "pbp2x_AAK99108.1_aligns.xml" para a sequência da proteína)

In [3]:
from Bio.Blast import NCBIXML
import re

files = [("pbpX",{}),("pbpX_AAK99108.1",{})]

for f in files:
    output1 = open(f"{f[0]}_aligns.xml", "w")

    result_handle = open(f"{f[0]}_blast.xml")

    blast_records = NCBIXML.parse(result_handle)

    for record in blast_records:

        first = ">QUERY\n"+record.alignments[0].hsps[0].query+"\n\n"
        f[1]["query"] = first
        
        for align in record.alignments:

            for hsp in align.hsps:

                match = re.sub(r"\w", "|", hsp.match)
                ###################
                #print ('****Alignment****')
                #print ('sequence:', align.title)
                #print ('ID:', align.hit_id)
                #print ('length:', align.length)
                #print ('e value:', hsp.expect)
                #print (hsp.query[0:30] + '...' + hsp.query[-30:] )
                #print( match[0:30] + '...' + match[-30:] )
                #print( hsp.sbjct[0:30] + '...' + hsp.sbjct[-30:] )
                #print("\n\n\n")
                ###################
                output1.write('****Alignment****\n')
                output1.write('sequence: '+ align.title+"\n")
                output1.write('ID: '+ align.hit_id+"\n")
                output1.write('length: '+ str(align.length)+"\n")
                output1.write('e value: '+ str(hsp.expect)+"\n")

                try:
                    try:
                        write = re.search(r"(gb|emb|sp|dbj)\|.+?\|", align.title).group()
                    except:
                        write = re.findall(r"\s.+?,", align.title)[0][1:-1].strip()
                        
                    f[1][write] = ">"+write+"\n"+hsp.sbjct+"\n\n"
                    
                except:
                    try:
                        write = re.search(r"(gb|emb|sp|dbj)\|.+?\|", align.title).group()
                    except:
                        write = re.findall(r"\s.+", align.title)[0][1:].strip()
                        
                    f[1][write] = ">"+write+"\n"+hsp.sbjct+"\n\n"

                st = 0
                for l in range(int(len(hsp.query)/100)+1):
                    position = st+100 if st+100<=len(hsp.query) else len(hsp.query)
                    output1.write(hsp.query[st:st+100]+f" {position}"+"\n")
                    output1.write(match[st:st+100]+"\n")
                    output1.write(hsp.sbjct[st:st+100]+"\n")
                    st += 100
                output1.write("\n\n")

    output1.close()
    result_handle.close()

for f in files:
    output2 = open(f"{f[0]}_homo.fasta", "w")
    order = list(f[1])
    order.sort()
    for o in order:
        if o == order[-1]:
            output2.write(re.sub(r"\ ", "_", f[1][o].strip()))
        else:
            output2.write(re.sub(r"\ ", "_", f[1][o]))
    output2.close()

O "output1" gera os ficheiros com os alinhamentos (1 com a sequência de DNA, outro com a sequência da proteína). O "output2" gera 2 ficheiros (na mesma lógica do que o "output1") apenas com as sequências homólogas encontradas em formato multi-FASTA.

### UniProt

Após ter transferido o ficheiro correspondente à proteína de interesse em formato txt, podemos processar esse ficheiro:

In [41]:
from Bio import ExPASy
import Bio.SwissProt as sp

with open("swissprot_pbp2x.txt") as handle:
    record = sp.read(handle)
    print("Nome:", record.entry_name,"\n")
    for acc in record.accessions:
        print("ID UniProt:", acc)
    print()
    for des in record.description.split(";"):
        print(des)
    print("Organismo:",repr(record.organism),"\n")
    print("Classificao do organismo:",record.organism_classification,"\n")
    print(record.sequence[:20] + "...","\n")
    print("Tamanho da seq:", record.sequence_length,"\n")
    print("Palavras Chave:",record.keywords,"\n")
    
counter = 1
for r in record.features:
    if r.type == "ACT_SITE":
        print(f"Sítio ativo {counter}:",r.location)
        counter += 1

Nome: PBPX_STRR6 

ID UniProt: P59676

RecName: Full=Penicillin-binding protein 2X
 Short=PBP-2X
 Short=PBP2X

Organismo: 'Streptococcus pneumoniae (strain ATCC BAA-255 / R6).' 

Classificao do organismo: ['Bacteria', 'Firmicutes', 'Bacilli', 'Lactobacillales', 'Streptococcaceae', 'Streptococcus'] 

MKWTKRVIRYATKNRKSPAE... 

Tamanho da seq: 750 

Palavras Chave: ['3D-structure', 'Antibiotic resistance', 'Cell cycle', 'Cell division', 'Cell membrane', 'Cell shape', 'Cell wall biogenesis/degradation', 'Membrane', 'Peptidoglycan synthesis', 'Reference proteome', 'Repeat', 'Transmembrane', 'Transmembrane helix'] 

Sítio ativo 1: [336:337]


A UniProt aponta para 1 sítio ativo, indicando a sua localização no gene na posição [336:337], que, pela literatura, sabemos que é da sua atividade de **transpeptidase**.